In [91]:
# import sqlite3
# import pandas as pd
# db_url = '/Users/huajiezhang/test/jupyter-collection/news-collection.db'

# db = sqlite3.connect(db_url)
from datetime import datetime, timedelta
dt_f = datetime.now() - timedelta(hours=48)
f = dt_f.strftime('%Y-%m-%d %H:%M:%S.%f')
df = pd.read_sql_query("SELECT news_title FROM news" 
                       "  where news_grab_ts > '{}'"
                       " and news_source <> 'Bloomberg'".format(f), db)

In [ ]:
df

In [88]:
# dt_f = datetime.now() - timedelta(hours=2)
# f = dt_f.strftime('%Y-%m-%d %H:%M:%S.%f')
# f

'2020-01-07 20:35:51.812800'

In [ ]:
import unidecode
import string
puncs = string.punctuation
maps = str.maketrans('','', puncs)
import nltk
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
import re
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words = [i.translate(maps) for i in stop_words]
import json

n_grms = ['south china morning post', 'hong kong', 'new york time', 'new york', 'stock market',
         'social security', 'credit card', 'hedge fund', 'wall street', 'jp morgan',
         'mutual fund']

def string_sub(s):
    for i in n_grms:
        s = re.sub(i, i.translate({ord(c): None for c in string.whitespace}), s)
    return s

not_need_in_results = ['south china morning post', 'bloomberg', 'new york time', 'bloomberg daybreak', 'stock',
                      'price','daybreak', 'wall street', 'buy', 'year', 'share', 'market', 'could', 'investor',
                      'show', 'stock market', 'company', 'today', 'analyst', 'say', 'top', 'target', 'time',
                      'week','new',]
not_need_in_results_c = [i.translate({ord(c): None for c in string.whitespace}) for i in not_need_in_results]


long_text = ' '.join(df['news_title'])
text = unidecode.unidecode(long_text)
text = text.translate(maps).lower()
wd_token = nltk.word_tokenize(text)
wd_token = [lemm.lemmatize(i) for i in wd_token]
wd_token = [i for i in wd_token if i.isalpha()]
long_string = ' '.join(wd_token)
long_string = string_sub(long_string)
wd_token = [i for i in nltk.word_tokenize(long_string) if i not in stop_words]

a = nltk.FreqDist(wd_token).most_common(300)
results = {i[0]:i[1] for i in a if not len(i[0])==1 if i[0] not in not_need_in_results_c}

with open('../data.json', 'w') as fp:
    json.dump(results, fp)

### steps
given a long string
- rm puncs, string level
- lower case, string level
- stem first list level 
- rm numbers, list level

- substitue terms, string level

- rm stop words, list level



In [82]:
# stop_words = [i.translate(maps) for i in stop_words]
# nltk_text = nltk.Text(wd_token)
nltk_text.concordance('money')

In [219]:
import json

with open('data.json', 'w') as fp:
    json.dump(dout, fp)

In [ ]:
with open('data.json') as f:
    data = json.load(f)
data

In [213]:
def nltk_freq_to_df(nltk_freq):
    d = {' '.join(k):[nltk_freq.get(k)] for k in nltk_freq}
    wd_df = pd.DataFrame.from_dict(d).transpose()
    wd_df = wd_df.reset_index()
    wd_df.columns = ['words', 'freq']
    wd_df = wd_df.sort_values(by='freq',ascending=False)
    
    return wd_df

u_df = nltk_freq_to_df(uni_freq)